In [78]:
#import the libraries
import pandas as pd
import sqlite3

In [79]:
# Read the data in json format
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [80]:
# Read the data in csv format
orders=pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [82]:
#import sqlite3 and create in-memory database
import sqlite3

conn = sqlite3.connect(":memory:")


In [83]:
#open and read the sql file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()


In [84]:
# Run all SQL commands in sql_script on the database connection
conn.executescript(sql_script)


In [85]:
# Load all data from the restaurants table into a Pandas DataFrame and display the first 5 rows by default

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [86]:
# Print the first 12 lines of the SQL script to inspect its contents
print(sql_script.split("\n")[:12])


['CREATE TABLE restaurants (', 'restaurant_id INT,', 'restaurant_name TEXT,', 'cuisine TEXT,', 'rating FLOAT);', '', "INSERT INTO restaurants VALUES (1, 'Restaurant_1', 'Chinese', 4.8);", "INSERT INTO restaurants VALUES (2, 'Restaurant_2', 'Indian', 4.1);", "INSERT INTO restaurants VALUES (3, 'Restaurant_3', 'Mexican', 4.3);", "INSERT INTO restaurants VALUES (4, 'Restaurant_4', 'Chinese', 4.1);", "INSERT INTO restaurants VALUES (5, 'Restaurant_5', 'Chinese', 4.8);", "INSERT INTO restaurants VALUES (6, 'Restaurant_6', 'Chinese', 4.8);"]


In [87]:
#display the first 5 rows from restaurants
restaurants = pd.read_sql("SELECT * FROM restaurants",conn)
restaurants.head()



,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [88]:
# Merge the orders and users datasets on 'user_id' using a left join
orders_users = pd.merge(orders,users,on="user_id",how="left")
orders_users.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [89]:
# Merge the orders_users dataframe with restaurants on 'restaurant_id' using a left join
final_df = pd.merge(orders_users,restaurants,on="restaurant_id",how="left")
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [90]:
#save the final joined dataset as  final_food_delivery_dataset.csv
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [91]:
## Helps verify that all orders are retained after merging with users and restaurants
print("Orders:", orders.shape[0])
print("Final Dataset:", final_df.shape[0])


Orders: 10000
Final Dataset: 10000


In [92]:
# Helps identify incomplete data after merging orders, users, and restaurants
final_df.isnull().sum()


order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

In [93]:
# Helps identify orders without matching user or restaurant records
final_df["user_id"].isna().sum(), final_df["restaurant_id"].isna().sum()


(np.int64(0), np.int64(0))

In [94]:
# Helps check if any orders are duplicated after merging
final_df["order_id"].nunique(), final_df.shape[0]


(10000, 10000)

In [95]:
# To ensure that merging did not change the total order amounts
orders["total_amount"].sum(), final_df["total_amount"].sum()


(np.float64(8011624.12), np.float64(8011624.12))

In [96]:
# Helps verify that all orders are retained in the merged dataset
orders["order_id"].min(), final_df["order_id"].min()


(np.int64(1), np.int64(1))

In [30]:
final_df.sample(5)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
5366,5367,7,302,28-10-2023,926.43,Spice Curry House Andhra,User_7,Chennai,Gold,Restaurant_302,Mexican,4.9
5673,5674,1143,296,13-02-2023,1498.11,Grand Restaurant Multicuisine,User_1143,Pune,Regular,Restaurant_296,Italian,3.8
4919,4920,2363,96,04-03-2023,651.07,Annapurna Biryani Non-Veg,User_2363,Bangalore,Regular,Restaurant_96,Mexican,4.4
342,343,1080,111,31-07-2023,960.54,Ruchi Dhaba Chinese,User_1080,Chennai,Regular,Restaurant_111,Mexican,4.2
5202,5203,1569,277,11-03-2023,259.66,Swagath Restaurant North Indian,User_1569,Pune,Gold,Restaurant_277,Mexican,3.3


In [35]:
audit = {
    "Total Orders": len(orders),
    "Final Rows": len(final_df),
    "Unique Orders": final_df["order_id"].nunique(),
    "Missing Restaurants": final_df["restaurant_name_x"].isna().sum(),
    "Revenue Match": orders["total_amount"].sum() == final_df["total_amount"].sum()
}

audit


{'Total Orders': 10000,
 'Final Rows': 10000,
 'Unique Orders': 10000,
 'Missing Restaurants': np.int64(0),
 'Revenue Match': np.True_}

In [97]:
# To identify the city with the highest revenue among Gold members
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue.idxmax()



'Chennai'

In [42]:
cuisine_avg_order = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

cuisine_avg_order


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [98]:
# Count the number of users whose total spending exceeds 1000
high_value_users = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)
count_users = (high_value_users > 1000).sum()
count_users


np.int64(2544)

In [101]:
# Create rating bins
rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=rating_bins,
    labels=rating_labels,
    include_lowest=True
)


In [102]:
rating_revenue = (
    final_df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


<ipython-input-102-117bef16b1bf>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

In [46]:
rating_revenue.idxmax()


'4.6 – 5.0'

In [103]:
# Calculate the average order value (AOV) for Gold members in each city
gold_city_aov = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
gold_city_aov.idxmax()



'Chennai'

In [104]:
# Sort the cuisines by the number of restaurants in ascending order
restaurants_per_cuisine = (
    final_df
    .groupby("cuisine")["restaurant_name_y"]
    .nunique()
    .sort_values()
)
restaurants_per_cuisine


cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_name_y, dtype: int64

In [105]:
# Sort cuisines by revenue in descending order
revenue_per_cuisine = (
    final_df
    .groupby("cuisine")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
revenue_per_cuisine


cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

In [54]:
cuisine_summary = (
    final_df
    .groupby("cuisine")
    .agg(
        distinct_restaurants=("restaurant_name_y", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .sort_values(by="distinct_restaurants")
)
cuisine_summary


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [107]:
# Select the top cuisine after sorting
(
    final_df
    .groupby("cuisine")
    .agg(
        distinct_restaurants=("restaurant_name_y", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .sort_values(
        by=["distinct_restaurants", "total_revenue"],
        ascending=[True, False]
    )
    .head(1)
)


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65


In [106]:
# Count the number of orders placed by Gold members
gold_orders = (final_df["membership"] == "Gold").sum()
total_orders = len(final_df)

percentage = (gold_orders / total_orders) * 100
round(percentage)


50

In [60]:
# Compute average order value & total orders by restaurant_name_x
restaurant_stats = (
    final_df
    .groupby("restaurant_name_x")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

# Filter restaurants with less than 20 orders
high_aov_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]

# Sort by average order value descending
answer = high_aov_restaurants.sort_values("avg_order_value", ascending=False).head(1)

answer


,total_orders,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308


In [64]:
restaurants_to_check = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]

# Filter only these restaurants
subset = final_df[final_df['restaurant_name_x'].isin(restaurants_to_check)]

# Group by restaurant, count orders, calculate average order value
summary = subset.groupby('restaurant_name_x').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
).reset_index()

# Keep only restaurants with less than 20 orders
filtered = summary[summary['total_orders'] < 20]

# Sort by avg_order_value descending and pick top
result = filtered.sort_values('avg_order_value', ascending=False).head(1)

print(result)


     restaurant_name_x  total_orders  avg_order_value
2  Ruchi Foods Chinese            19       686.603158


In [66]:
final_df.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [68]:
# Filter the dataset for only the relevant combinations
options = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]

# Create a new column to identify the combination
final_df['combo'] = final_df['membership'] + ' + ' + final_df['cuisine']

# Filter only the four options
filtered_df = final_df[final_df['combo'].isin([f"{m} + {c}" for m, c in options])]

# Group by combo and sum total_amount
revenue_by_combo = filtered_df.groupby('combo')['total_amount'].sum().reset_index()

# Sort descending to get the highest revenue
revenue_by_combo = revenue_by_combo.sort_values(by='total_amount', ascending=False)

print(revenue_by_combo)

# Top combination
top_combo = revenue_by_combo.iloc[0]
print("\nHighest revenue combination:")
print(top_combo)


               combo  total_amount
1     Gold + Italian    1005779.05
3   Regular + Indian     992100.27
0      Gold + Indian     979312.31
2  Regular + Chinese     952790.91

Highest revenue combination:
combo           Gold + Italian
total_amount        1005779.05
Name: 1, dtype: object


In [69]:
# Make sure order_date is datetime
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

# Extract the quarter from the order_date
final_df['quarter'] = final_df['order_date'].dt.quarter

# Group by quarter and sum total_amount
revenue_by_quarter = final_df.groupby('quarter')['total_amount'].sum().reset_index()

# Sort descending to find the highest revenue quarter
revenue_by_quarter = revenue_by_quarter.sort_values(by='total_amount', ascending=False)

print(revenue_by_quarter)

# Top quarter
top_quarter = revenue_by_quarter.iloc[0]
print(f"\nQuarter with highest revenue: Q{int(top_quarter['quarter'])} with total revenue {top_quarter['total_amount']}")


   quarter  total_amount
2        3    2037385.10
3        4    2018263.66
0        1    2010626.64
1        2    1945348.72

Quarter with highest revenue: Q3 with total revenue 2037385.1


In [70]:
# Count orders where membership is 'Gold'
gold_orders_count = final_df[final_df['membership'] == 'Gold'].shape[0]
print(f"Total orders by Gold members: {gold_orders_count}")


Total orders by Gold members: 4987


In [71]:
# Filter for Hyderabad orders and sum the revenue
hyderabad_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()

# Round to nearest integer
hyderabad_revenue_rounded = round(hyderabad_revenue)

print(f"Total revenue from Hyderabad: {hyderabad_revenue_rounded}")


Total revenue from Hyderabad: 1889367


In [72]:
# Count distinct users
distinct_users = final_df['user_id'].nunique()

print(f"Number of distinct users: {distinct_users}")


Number of distinct users: 2883


In [73]:
# Filter orders placed by Gold members
gold_orders = final_df[final_df['membership'] == 'Gold']

# Calculate average order value
avg_order_value_gold = round(gold_orders['total_amount'].mean(), 2)

print(f"Average order value for Gold members: {avg_order_value_gold}")


Average order value for Gold members: 797.15


In [74]:
# Filter orders where restaurant rating is >= 4.5
high_rating_orders = final_df[final_df['rating'] >= 4.5]

# Count the total number of such orders
total_high_rating_orders = high_rating_orders.shape[0]

print(f"Total orders for restaurants with rating ≥ 4.5: {total_high_rating_orders}")


Total orders for restaurants with rating ≥ 4.5: 3374


In [75]:
# Filter only Gold members
gold_orders = final_df[final_df['membership'] == 'Gold']

# Compute total revenue by city for Gold members
city_revenue = gold_orders.groupby('city')['total_amount'].sum()

# Find the city with the highest revenue
top_city = city_revenue.idxmax()

# Count the number of orders in that city
orders_in_top_city = gold_orders[gold_orders['city'] == top_city].shape[0]

print(f"Top revenue city among Gold members: {top_city}")
print(f"Number of orders in that city: {orders_in_top_city}")


Top revenue city among Gold members: Chennai
Number of orders in that city: 1337


In [76]:
final_df.shape[0]


10000